In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext()

spark = SparkSession.builder.appName("Python Spark DataFrames basic example").config("spark.some.config.option", "some-value").getOrCreate()

25/01/17 21:27:12 WARN Utils: Your hostname, sridhar resolves to a loopback address: 127.0.1.1; using 172.18.174.89 instead (on interface wlp1s0)
25/01/17 21:27:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/17 21:27:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/17 21:27:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
if 'spark' in locals() and isinstance(spark, SparkSession):
    print("SparkSession is active and ready to use.")
else:
    print("SparkSession is not active. Please create a SparkSession.")

SparkSession is active and ready to use.


In [5]:
data = range(1,30)
print(data[0])
len(data)
xrangeRDD = sc.parallelize(data, 4)

xrangeRDD

1


PythonRDD[1] at RDD at PythonRDD.scala:53

In [6]:
subRDD = xrangeRDD.map(lambda x:x-1)
filteredRDD = subRDD.filter(lambda x:x<10)

In [7]:
print(filteredRDD.collect())
filteredRDD.count()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


10

In [8]:
import time
test = sc.parallelize(range(1,50000), 4)
test.cache()

t1 = time.time()
count1 = test.count()
dt1 = time.time() - t1
print("dt1: ", dt1)

t2 = time.time()
count2 = test.count()
dt2 = time.time() - t2
print("dt2: ", dt2)

dt1:  0.272125244140625
dt2:  0.10077953338623047


In [9]:
spark

In [10]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/people.json >> people.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    73  100    73    0     0     40      0  0:00:01  0:00:01 --:--:--    40


In [11]:
df = spark.read.json("people.json").cache()

In [12]:
df.show()
df.printSchema()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [13]:
df.createTempView("people")

In [14]:
df.select("name").show()
df.select(df["name"]).show()
spark.sql("SELECT name FROM people").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
+-------+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
+-------+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
+-------+



In [15]:
df.filter(df["age"] > 21).show()
spark.sql("SELECT age, name FROM people WHERE age > 21").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
| 30|Andy|
+---+----+

+---+----+
|age|name|
+---+----+
| 30|Andy|
| 30|Andy|
+---+----+



In [16]:
df.groupBy("age").count().show()
spark.sql("SELECT age, COUNT(age) as count FROM people GROUP BY age").show()

+----+-----+
| age|count|
+----+-----+
|  19|    2|
|NULL|    2|
|  30|    2|
+----+-----+

+----+-----+
| age|count|
+----+-----+
|  19|    2|
|NULL|    0|
|  30|    2|
+----+-----+

